# 🧳 AI Travel Planner (LangChain + Gemini)

This notebook demonstrates a **Generative AI-powered travel planner** built using:

- **LangChain** for prompt templating & message management  
- **LangGraph** for workflow orchestration  
- **Gemini (Google Generative AI)** as the LLM  
- **Python + Jupyter** for interactive execution  

You'll be able to:
1. Input a city and your interests.
2. Generate a personalized **day-trip itinerary**.
3. Understand how to build a modular agent workflow with best practices.

---

## ⚙️ Setup & Installation

Before running, make sure you have:

```bash
pip install langchain langchain-core langchain-google-genai langgraph python-dotenv ipython


In [4]:

import os
from typing import TypedDict, Annotated, List

from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI


## 📂 Step 2: Define the Application State

We use a `TypedDict` to represent the state as it moves through the graph.
This makes the code more **type-safe** and easy to debug.


In [5]:
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation history"]
    city: str
    interests: List[str]
    itinerary: str



## 🔑 Step 3: Load Environment Variables

We load the `GOOGLE_API_KEY` from the `.env` file for security.


In [6]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found. Please add it to your .env file.")
print("✅ Environment variables loaded successfully")


✅ Environment variables loaded successfully


## 🤖 Step 4: Configure the Gemini LLM

We use **Gemini 2.5 Flash** with a moderate temperature for balanced creativity.


In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=google_api_key,
    temperature=0.7,
)

print("✅ LLM configured successfully")


✅ LLM configured successfully


## 🧾 Step 5: Define Prompt Template

We create a reusable **ChatPromptTemplate** to guide the LLM in generating itineraries.


In [8]:
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day trip for {city} based on user's interests: {interests}. Provide a brief, bulleted itinerary."),
    ("human", "Create an itinerary for my day trip"),
])


## 🔗 Step 6: Define Workflow Nodes

Each node represents a step in the workflow:

1. **input_city** → Ask user for the city.
2. **input_interest** → Ask user for their interests.
3. **create_itinerary** → Generate and display itinerary.


In [9]:
def input_city(state: PlannerState) -> PlannerState:
    user_message = input("Enter the city you want to visit: ")
    return {
        **state,
        "city": user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)],
    }

def input_interest(state: PlannerState) -> PlannerState:
    user_message = input(f"Enter your interests for {state['city']} (comma-separated): ")
    return {
        **state,
        "interests": [i.strip() for i in user_message.split(",")],
        "messages": state['messages'] + [HumanMessage(content=user_message)],
    }

def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"\n📝 Creating itinerary for {state['city']}...\n")
    response = llm.invoke(itinerary_prompt.format_messages(
        city=state["city"],
        interests=". ".join(state["interests"])
    ))
    print("✅ Suggested Itinerary:\n")
    print(response.content)
    return {
        **state,
        "messages": state['messages'] + [AIMessage(content=response.content)],
        "itinerary": response.content,
    }


## 🧠 Step 7: Build the LangGraph Workflow


In [10]:
workflow = StateGraph(PlannerState)

workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("create_itinerary", create_itinerary)

workflow.set_entry_point("input_city")
workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "create_itinerary")
workflow.add_edge("create_itinerary", END)

app = workflow.compile()
print("✅ Workflow compiled successfully")


✅ Workflow compiled successfully


## ▶️ Step 8: Run the Travel Planner

The `travel_planner` function initializes state and streams through the workflow.


In [11]:
def travel_planner(user_request: str):
    print(f"🌍 Initial Request: {user_request}\n")
    state = {
        "messages": [HumanMessage(content=user_request)],
        "city": "",
        "interests": [],
        "itinerary": "",
    }
    for _ in app.stream(state):
        pass

# Run the app interactively
travel_planner("I want to plan a trip")


🌍 Initial Request: I want to plan a trip


📝 Creating itinerary for Hyderabad...

✅ Suggested Itinerary:

Here's a fascinating day trip itinerary for Hyderabad, focusing on its rich historical heritage, well within your generous budget of ₹30,000!

**A Day of Historical Wonders in Hyderabad**

Embark on a journey through time, exploring the grand forts, majestic palaces, and ancient mosques that tell the tale of Hyderabad's glorious past.

*   **Morning (9:00 AM - 1:00 PM): Golconda's Grandeur & Royal Tombs**
    *   **9:00 AM - 11:30 AM: Golconda Fort:** Start your day at the magnificent **Golconda Fort**. Explore its impressive acoustics, climb to the top for panoramic views of the city, and imagine its diamond trading past. Allow ample time to wander through its various sections.
    *   **11:30 AM - 1:00 PM: Qutb Shahi Tombs:** Just a short drive from Golconda, visit the serene **Qutb Shahi Tombs**. These architectural marvels are the resting places of the Qutb Shahi rulers and the